# Exercises

1. What is the fundamental idea behind support vector machines?
2. What is a support vector?
3. Why is it important to scale the inputs when using SVMs?
4. Can a SVM classifier output a confidence score when it classifies an instance? What about a probability?
5. Should you use the primal or dual form of the SVM problem to train a model on a training set with millions of instances & hundred of features?
6. Say you trained an SVM classifier with an RBF kernel. it seems to underfit the training set: should you increase or decrease $\gamma$ (`gamma`)? What about `C`?
7. How should you set the QP parameters ($H$, $f$, $A$, & $b$) to solve the soft margin linear SVM classifier problem using an off-the-shelf QP solver?
8. Train a `LinearSVC` on a linearly separable dataset. Then train an `SVC` & a `SGDClassifer` on the same dataset. See if you can get them to produce roughly the same model.
9. Train an SVM classifier on the MNIST dataset. Since SVM classifiers are binary classifiers, you will need to use one-versus-all to classify all 10 digits. you may want to tune the hyperparameters using small validation sets to speed up the process. What accuracy can you reach?
10. Train an SVM regressor on the California housing dataset.

---